# 隐藏代码 

In [12]:
# -*- coding: cp936 -*-
import pandas as pd
#from bs4 import BeautifulSoup
import urllib,math
import requests
#from pymongo import MongoClient
#import cgi
import re,datetime,json,time
from math import  floor

from sklearn import metrics
from sklearn import preprocessing, cross_validation, svm
from sklearn.model_selection import train_test_split   #这里是引用了交叉验证 
from sklearn.linear_model import LinearRegression
import json,csv

import statsmodels.api as sm
import numpy as np
import pandas as pd
import urllib.request



maxTryNum=10
url0='http://www.51tradecloud.com/'

#1.收票/出票
url = url0+'g/301'
for tries in range(maxTryNum):
  try:
      res_data = urllib.request.urlopen(url,timeout=25)
      res_data = res_data.read()
      res_data=res_data.decode('utf-8')
      piaofenxi= pd.read_json(res_data,typ='frame')  
      break
  except:
      if tries < (maxTryNum - 1):
          continue
      else:
          piaofenxi=pd.DataFrame({})
          break
#print(piaofenxi)
piaofenxi['统计日期'] = pd.to_datetime(piaofenxi['统计日期']).astype('str')     #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式
piaofenxi=piaofenxi[piaofenxi['机构']!='中介']
piaofenxi1=piaofenxi['收票'].groupby([piaofenxi['统计日期']]).sum()
piaofenxi2=piaofenxi['出票'].groupby([piaofenxi['统计日期']]).sum()
piaofenxi2=piaofenxi2.reset_index(drop = False)
piaofenxi1=piaofenxi1.reset_index(drop = False)
piaofenxi1=piaofenxi1.set_index('统计日期')
piaofenxi2=piaofenxi2.set_index('统计日期')
piaofenxi3=pd.concat([piaofenxi1,piaofenxi2], axis=1)


#2.票交所发行量
url = url0+'y/401'
for tries in range(maxTryNum):
  try:
      res_data = urllib.request.urlopen(url,timeout=25)
      res_data = res_data.read()
      res_data=res_data.decode('utf-8')
      piaojiaosuo= pd.read_json(res_data,typ='frame')  
      break
  except:
      if tries < (maxTryNum - 1):
          continue
      else:
          piaojiaosuo=pd.DataFrame({})
          break
piaojiaosuo['提取日期'] = pd.to_datetime(piaojiaosuo['提取日期']).astype('str')   #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式 注意要换成string格式
piaojiaosuo = piaojiaosuo.sort_values(by=['提取日期'], ascending=True)
piaojiaosuo2=piaojiaosuo[piaojiaosuo['数据']=='当日']
piaojiaosuo2=piaojiaosuo2[['提取日期','承兑金额/亿元','贴现金额/亿元','交易金额/亿元','承兑余额/亿元','贴现余额/亿元']]
piaojiaosuo2=piaojiaosuo2.drop_duplicates('提取日期', keep='last')  #删除重复值
model_piaojiaosuo_df=piaojiaosuo2.set_index('提取日期')










#3.票交所收益率曲线
url = url0+'y/201'
for tries in range(maxTryNum):
  try:
      res_data = urllib.request.urlopen(url,timeout=25)
      res_data = res_data.read()
      res_data=res_data.decode('utf-8')
      paojiaosuoll= pd.read_json(res_data,typ='frame')  
      break
  except:
      if tries < (maxTryNum - 1):
          continue
      else:
          paojiaosuoll=pd.DataFrame({})
          break
paojiaosuoll['日期'] = pd.to_datetime(paojiaosuoll['日期']).astype('str')   #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式 注意要换成string格式
paojiaosuoll = paojiaosuoll.sort_values(by=['日期'], ascending=True)
paojiaosuoll=paojiaosuoll.set_index('日期')


#4.生成银行存单
url = url0+'y/101'
for tries in range(maxTryNum):
  try:
      res_data = urllib.request.urlopen(url,timeout=25)
      res_data = res_data.read()
      res_data=res_data.decode('utf-8')
      result0= pd.read_json(res_data,typ='frame')  
      break
  except:
      if tries < (maxTryNum - 1):
          continue
      else:
          result0=pd.DataFrame({})
          break
result0= result0.reset_index(drop=True)    #重新定义索引
result0=result0[result0['银行分类']!='外资行']
result0= result0.reset_index(drop=True)    #重新定义索引
print(result0)
#result0.to_excel('ee.xls')

guogu=['工商银行','农行','建行','交通银行','邮储银行','中国银行']
gufen=['光大银行','广发银行','民生银行','浦发银行','平安银行','兴业银行','中信银行','招商银行','华夏']
for i in range(0, len(result0)):
        if (result0.loc[i,'银行分类'] in guogu):
            result0.loc[i,'银行分类2']='国股行'

        elif(result0.loc[i,'银行分类']in gufen):
            result0.loc[i,'银行分类2']='股份行'

        elif(result0.loc[i,'银行分类']=='城商'):
            result0.loc[i,'银行分类2']='城商'

        elif(result0.loc[i,'银行分类']=='农商行'):
            result0.loc[i,'银行分类2']='农商行'
result0['shi']=result0['实际发行(亿)']*result0['实际加权利率(%)AAA']
result0['ji']=result0['计划发行(亿)']*result0['计划加权利率(%)AAA']


result0=result0.groupby(['xz','发行日','期限','银行分类2']).sum()  #在这一步，空白的开始变成数值0
result0['实际加权利率2(%)AAA']=result0['shi']/result0['实际发行(亿)']
result0['计划加权利率2(%)AAA']=result0['ji']/result0['计划发行(亿)']


re11=result0.query('xz==1')
re11=re11.reset_index(["xz"],drop=True)
re11=re11[['实际发行(亿)','计划发行(亿)','实际加权利率2(%)AAA','计划加权利率2(%)AAA']]

re22=result0.query('xz==2')
re22=re22.reset_index(["xz"],drop=True)
re22=re22[['计划加权利率2(%)AAA','计划发行(亿)']]
re22.rename(columns={'计划发行(亿)': '原计划发行(亿)','计划加权利率2(%)AAA':'原计划加权利率2(%)AAA'}, inplace=True) 


result0 = pd.concat([re11, re22], axis=1)
result0=result0.reset_index(drop=False)


#for i in range(0,len(result0)):
#        if result0.loc[i,'原计划加权利率2(%)AAA']==0:
#               result0.loc[i,'原计划加权利率2(%)AAA'] =np.nan
#        if result0.loc[i,'实际加权利率2(%)AAA']==0:
#               result0.loc[i,'实际加权利率2(%)AAA'] =np.nan



model_cundan_df=result0[result0['银行分类2']=='股份行']
model_cundan_df=model_cundan_df[['发行日','期限','实际加权利率2(%)AAA','原计划加权利率2(%)AAA']]
model_cundan_df=model_cundan_df.reset_index(drop=False)

model_cundan_df=model_cundan_df.fillna(value='')  #先把NAN提成0

for i in range(0,len(model_cundan_df)):
    if (model_cundan_df.loc[i,'实际加权利率2(%)AAA']=='')and( (model_cundan_df.loc[i,'原计划加权利率2(%)AAA'] !=0 )):
 #       print('22')
        model_cundan_df.loc[i,'实际加权利率2(%)AAA']= model_cundan_df.loc[i,'原计划加权利率2(%)AAA']
model_cundan_df=model_cundan_df[model_cundan_df['实际加权利率2(%)AAA']!='']        
print(model_cundan_df)        
#model_cundan_df['实际加权利率2(%)AAA']=model_cundan_df['实际加权利率2(%)AAA'].astype('float')

#stop        
model_cundan_df.to_excel('3e.xls')
        
model_cundan_df=model_cundan_df[['发行日','期限','实际加权利率2(%)AAA']]
model_cundan_df['实际加权利率2(%)AAA'] =model_cundan_df['实际加权利率2(%)AAA'].astype('float')   #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式 注意要换成string格式


model_cundan_df['发行日'] = pd.to_datetime(model_cundan_df['发行日']).astype('str')   #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式 注意要换成string格式
model_cundan_df=model_cundan_df.set_index(['发行日'],drop=True)

cundan1=model_cundan_df[model_cundan_df['期限']=='1月']
cundan2=model_cundan_df[model_cundan_df['期限']=='3月']
cundan3=model_cundan_df[model_cundan_df['期限']=='6月']
cundan4=model_cundan_df[model_cundan_df['期限']=='1年']
model_cundan_df=pd.concat([cundan1,cundan2,cundan3,cundan4], axis=1,join='outer') #默认是outer

model_cundan_df['1个月存单利率']=model_cundan_df.iloc[:,1]
model_cundan_df['3个月存单利率']=model_cundan_df.iloc[:,3]
model_cundan_df['6个月存单利率']=model_cundan_df.iloc[:,5]
model_cundan_df['1年存单利率']=model_cundan_df.iloc[:,7]
model_cundan_df=model_cundan_df[['1个月存单利率','3个月存单利率','6个月存单利率','1年存单利率']]


print(model_cundan_df)

#stop
#4.线上同存价格
url = url0+'y/601'
for tries in range(maxTryNum):
  try:
      res_data = urllib.request.urlopen(url,timeout=25)
      res_data = res_data.read()
      res_data=res_data.decode('utf-8')
      model_tongcun_df= pd.read_json(res_data,typ='frame')  
      break
  except:
      if tries < (maxTryNum - 1):
          continue
      else:
          model_tongcun_df=pd.DataFrame({})
          break
model_tongcun_df=model_tongcun_df[['日期','kind','加权利率(%)','收盘利率(%)']]
model_tongcun_df['日期'] = pd.to_datetime(model_tongcun_df['日期']).astype('str')   #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式 注意要换成string格式

zhiya1=model_tongcun_df[model_tongcun_df['kind']=='IBO001']
zhiya1=zhiya1.drop_duplicates('日期', keep='last')  #删除重复值
zhiya1=zhiya1.set_index('日期')

zhiya2=model_tongcun_df[model_tongcun_df['kind']=='IBO007']
zhiya2=zhiya2.drop_duplicates('日期', keep='last')  #删除重复值
zhiya2=zhiya2.set_index('日期')
model_tongcun_df=pd.concat([zhiya1,zhiya2], axis=1,join='outer') #默认是outer

model_tongcun_df['隔夜同存加权利率']=model_tongcun_df.iloc[:,1]
model_tongcun_df['隔夜同存收盘利率']=model_tongcun_df.iloc[:,2]
model_tongcun_df['7天同存加权利率']=model_tongcun_df.iloc[:,4]
model_tongcun_df['7天同存收盘利率']=model_tongcun_df.iloc[:,5]
model_tongcun_df=model_tongcun_df[['隔夜同存加权利率','隔夜同存收盘利率','7天同存加权利率','7天同存收盘利率']]




#5.质押式回购

url = url0+'y/501'
for tries in range(maxTryNum):
  try:
      res_data = urllib.request.urlopen(url,timeout=25)
      res_data = res_data.read()
      res_data=res_data.decode('utf-8')
      model_zhiya_df= pd.read_json(res_data,typ='frame')  
      break
  except:
      if tries < (maxTryNum - 1):
          continue
      else:
          model_zhiya_df=pd.DataFrame({})
          break
model_zhiya_df=model_zhiya_df[['日期','kind','加权利率(%)','收盘利率(%)']]
model_zhiya_df['日期'] = pd.to_datetime(model_zhiya_df['日期']).astype('str')   #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式 注意要换成string格式

zhiya1=model_zhiya_df[model_zhiya_df['kind']=='R001']
zhiya1=zhiya1.drop_duplicates('日期', keep='last')  #删除重复值
zhiya1=zhiya1.set_index('日期')

zhiya2=model_zhiya_df[model_zhiya_df['kind']=='R007']
zhiya2=zhiya2.drop_duplicates('日期', keep='last')  #删除重复值
zhiya2=zhiya2.set_index('日期')
model_zhiya_df=pd.concat([zhiya1,zhiya2], axis=1,join='outer') #默认是outer

model_zhiya_df['隔夜质押加权利率']=model_zhiya_df.iloc[:,1]
model_zhiya_df['隔夜质押收盘利率']=model_zhiya_df.iloc[:,2]
model_zhiya_df['7天质押加权利率']=model_zhiya_df.iloc[:,4]
model_zhiya_df['7天质押收盘利率']=model_zhiya_df.iloc[:,5]
model_zhiya_df=model_zhiya_df[['隔夜质押加权利率','隔夜质押收盘利率','7天质押加权利率','7天质押收盘利率']]


#二、汇总成一个表
yuanshi_df2=pd.concat([model_cundan_df,piaofenxi3,model_zhiya_df,model_piaojiaosuo_df,paojiaosuoll,model_tongcun_df], axis=1,join='outer')
yuanshi_df2=yuanshi_df2.reset_index(drop = False)
yuanshi_df2.rename(columns={'index':'日期'}, inplace=True)
yuanshi_df2=yuanshi_df2[['日期','3个月存单利率','6个月存单利率','1年存单利率','1个月存单利率',
               '12月','7天质押加权利率',
               '隔夜质押加权利率','7天同存加权利率','隔夜同存加权利率','承兑金额/亿元','贴现金额/亿元','收票','出票']]  #'收票/出票','贴/承b','贴/交b','7天质押加权利率',
yuanshi_df2.rename(columns={'12月':'足年国股'}, inplace=True)
yuanshi_df2= yuanshi_df2.reset_index(drop=True)    #重新定义索引

#########



echartguogu=yuanshi_df2[['日期','1个月存单利率','3个月存单利率','6个月存单利率','1年存单利率','足年国股' ]]
echartguogu=echartguogu.set_index('日期')
echartguogu=echartguogu.dropna(how='all')
echartguogu=echartguogu.fillna(method='ffill')  #用缺失值前面的数字填充缺失值

echartguogu= echartguogu.reset_index(drop=False)    #重新定义索引

echartgg=echartguogu['日期'].tolist()
echartgg1=echartguogu['1个月存单利率'].tolist()
echartgg2=echartguogu['3个月存单利率'].tolist()
echartgg3=echartguogu['6个月存单利率'].tolist()
echartgg4=echartguogu['1年存单利率'].tolist()
echartgg5=echartguogu['足年国股'].tolist()






#二、模型

guogu_df=yuanshi_df2[['日期','6个月存单利率','足年国股','7天质押加权利率','7天同存加权利率','承兑金额/亿元','贴现金额/亿元','收票','出票']]
print(guogu_df.tail())
guogu_df['日期'] = pd.to_datetime(guogu_df['日期']).astype('str')   #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式
guogu_df['6个月存单利率']=guogu_df['6个月存单利率'].astype('float')
guogu_df['7天同存加权利率']=guogu_df['7天同存加权利率'].astype('float')
guogu_df['7天质押加权利率']=guogu_df['7天质押加权利率'].astype('float')

guogu_df['收票/出票']=guogu_df['收票'].astype('float')/guogu_df['出票'].astype('float')
guogu_df['贴/承']=guogu_df['贴现金额/亿元'].astype('float')/guogu_df['承兑金额/亿元'].astype('float')
guogu_df['足年国股']=guogu_df['足年国股'].astype('float')
guogu_df['6个月存单利率b']=guogu_df['6个月存单利率'].shift(1)
guogu_df['贴/承b']=guogu_df['贴/承'].shift(1)
guogu_df['足年国股f']=guogu_df['足年国股'].shift(-1)
guogu_df['日期f']=guogu_df['日期'].shift(-1)



#1.1.取出最新数据预备作线性回归预测数据
guogu_df2=guogu_df.fillna(method='ffill')  
x1=guogu_df2.loc[len(guogu_df2)-1,'6个月存单利率']
x2= guogu_df2.loc[len(guogu_df2)-1,'7天质押加权利率']
x3=  guogu_df2.loc[len(guogu_df2)-1,'7天同存加权利率']
d4=  guogu_df2.loc[len(guogu_df2)-1,'日期']    

#1.2.取出最新数据预备作KNN预测数据
guogu_df2['6个月存单利率bq'] = guogu_df2['6个月存单利率']-guogu_df2['6个月存单利率b'] 
x21=guogu_df2.loc[len(guogu_df2)-1,'6个月存单利率bq']
x22= guogu_df2.loc[len(guogu_df2)-1,'收票/出票']
x23=  guogu_df2.loc[len(guogu_df2)-1,'贴/承b']


#for i in range(len(guogu_df)):
#    if np.isnan(guogu_df['足年国股'][i]) :
#        guogu_df = guogu_df.drop(i)




#guogu_df= guogu_df.reset_index(drop=True)    #重新定义索引

#删除最后1条数据
guogu_df = guogu_df[:len(guogu_df)-2]
guogu_df= guogu_df.reset_index(drop=True)    #重新定义索引





# 二、做线性回归
yuanshi_df=guogu_df[['日期','日期f','6个月存单利率b',
       '足年国股f',#'收票/出票',

      '7天质押加权利率', '7天同存加权利率'
      ]]  #'收票/出票','贴/承b','贴/交b','7天质押加权利率',

# print(yuanshi_df)

yuanshi_df=yuanshi_df.fillna(method='ffill')  #用缺失值前面的数字填充缺失值

#2.1排除月末4-5天的数据
for i in range(len(yuanshi_df)):
  if (yuanshi_df['日期'][i])[len(yuanshi_df['日期'][i])-2:]  in ['30','31','29','28','27']:
     yuanshi_df = yuanshi_df.drop(i)


yuanshi_df=yuanshi_df.dropna(how='any')


x=yuanshi_df[[ '6个月存单利率b',
     '7天质押加权利率',
'7天同存加权利率'
     ]]  
X = sm.add_constant(x)
beta=np.array([1,10])
e = np.random.normal(size=len(yuanshi_df))
y=yuanshi_df[ '足年国股f']


#2.2构建模型
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split   #这里是引用了交叉验证 
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state=1)
linreg=linear_model.LinearRegression()
model=linreg.fit(X_train, y_train)#yh=lr.predict(x)  #.reshape(-1,1)
confidence=round(linreg.score(X_test, y_test),2)  #查看测试模型的准确率



#六、模型画图
y_pred = linreg.predict(X)




yuanshi_df['预测足年国股']=y_pred

yuce_df=pd.DataFrame()


yuanshi_df=yuanshi_df[['足年国股f','预测足年国股','日期','日期f']]
w=[[1,x1,x2,x3]]
#w= sm.add_constant(w)
y_pred7 = linreg.predict(w)
yuce_df.loc[0,'日期f']=d4
yuce_df.loc[0,'预测足年国股']=y_pred7

zong_df=pd.concat([yuanshi_df,yuce_df])

#  print(zong_df.tail())





#二、模型二
guogu_df=guogu_df[['贴/承b','出票','日期', '日期f','6个月存单利率','6个月存单利率b','足年国股f','足年国股','收票/出票']]
guogu_df['足年国股fq2']=0

guogu_df=guogu_df.fillna(method='ffill')  #用缺失值前面的数字填充缺失值

guogu_df['足年国股fq']= guogu_df['足年国股f']-guogu_df['足年国股']  ###########

for i in range(len(guogu_df)):
    if guogu_df['足年国股fq'][i]>0:
        guogu_df['足年国股fq2'][i]=1 
    else:
        guogu_df['足年国股fq2'][i]=0






guogu_df= guogu_df.reset_index(drop=True)    #重新定义索引

guogu_df['6个月存单利率bq'] = guogu_df['6个月存单利率']-guogu_df['6个月存单利率b'] 
guogu_df=guogu_df[guogu_df['出票'] !=0]
guogu_df=guogu_df.dropna(how='any')
guogu_df= guogu_df.reset_index(drop=True)    #重新定义索引

X=guogu_df[[  
   '贴/承b',
    '6个月存单利率bq',
    '收票/出票']]
y=guogu_df['足年国股fq2']
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split   #这里是引用了交叉验证 
#X_train,X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state=1)
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size = 0.2,random_state=1)






knn_classifier=KNeighborsClassifier(12)
knn_classifier.fit(x_train,y_train)
y_predict=knn_classifier.predict(x_test)
scores=knn_classifier.score(x_test,y_test)
#   print('acc:{}'.format(sum(y_predict==y_test)/len(x_test)),scores)
confidence2=round(sum(y_predict==y_test)/len(x_test),2)


y_predict=knn_classifier.predict(X)



guogu_df['预测涨跌']=y_predict



guogu_df=guogu_df[['预测涨跌','足年国股fq2','日期', '日期f']]

#w= sm.add_constant(w)
yuce_df2=pd.DataFrame()
w=[[x21,x22,x23]]

yuce_df2.loc[0,'日期f']=d4
yuce_df2.loc[0,'预测涨跌']=knn_classifier.predict(w)




#   print(yuce_df2)

zong_df2=pd.concat([guogu_df,yuce_df2])
zong_df=zong_df.set_index(['日期f'])
del zong_df['日期']
zong_df2=zong_df2.set_index(['日期f'])
del zong_df2['日期']



#三、将两个模型的预测数据合并，做图
zong_df3=pd.concat([zong_df,zong_df2],axis=1)


zong_df3= zong_df3.reset_index(drop=False)    #重新定义索引
zong_df3=zong_df3.fillna('')   #把所有空值统一成''


zong_df3['预测涨跌2']=''

zong_df3.loc[len(zong_df3)-1,'预测涨跌2']=zong_df3.loc[len(zong_df3)-1,'预测足年国股']




zong_df3=zong_df3[zong_df3['预测足年国股']!='']
zong_df3= zong_df3.reset_index(drop=True)    #重新定义索引

yuce2=zong_df3['预测涨跌2'].tolist()
yuce3=str(zong_df3.loc[len(zong_df3)-1,'预测涨跌'])
print(zong_df3)
#     print(yuce3)
date_shun=zong_df3['index'].tolist()
yuce=zong_df3['预测足年国股'].tolist()
shiji=zong_df3['足年国股f'].tolist()

zuixiao=round(min(yuce+zong_df3['足年国股f'][:len(zong_df3)-1].astype('float').tolist()),2)
# zong_df3=zong_df3.set_index('index').T.to_dict('list')




x1=round(x1,2)
x2=round(x2,2)
x3=round(x3,2)
x21=round(x21,2)
x22=round(x22,2)
x23=round(x23,2)


       xz         发行日  实际加权利率(%)  实际加权利率(%)AAA  实际发行(亿)  期限  条数  计划加权利率(%)  \
0       1  2018-01-02       4.00          4.00     50.0  1月   1       4.00   
1       1  2018-01-02       4.70          4.70     24.7  3月   1       4.70   
2       1  2018-01-03       4.00          4.00     39.2  1月   1       4.00   
3       1  2018-01-08       4.70          4.70     61.6  3月   4       4.70   
4       1  2018-01-31       4.80          4.80     21.0  3月   1       4.80   
5       2  2018-10-12        NaN           NaN      NaN  9月   6       3.58   
6       2  2018-10-12        NaN           NaN      NaN  9月   1       3.50   
7       2  2018-10-12        NaN           NaN      NaN  9月   1       3.45   
8       2  2018-10-12        NaN           NaN      NaN  9月   1       3.50   
9       2  2018-10-12        NaN           NaN      NaN  9月   1       3.50   
10      2  2018-10-12        NaN           NaN      NaN  9月   1       3.55   
11      2  2018-10-12        NaN           NaN      NaN  9月   1 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:199: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



             1个月存单利率   3个月存单利率   6个月存单利率    1年存单利率
2018-01-02  4.000000  4.624547  4.800000       NaN
2018-01-03  4.000000  4.600000  4.600000  4.800000
2018-01-04  3.800000  4.600000  4.600000  4.848673
2018-01-05  3.600000  4.618371  4.600000  4.850000
2018-01-08  3.400000  4.700000  4.700000  4.850000
2018-01-09  3.200000  4.708856       NaN  4.850000
2018-01-10       NaN  4.731893  4.679365  4.845925
2018-01-11  3.200000  4.721886  4.700000  4.850000
2018-01-12  3.500000  4.750000  4.750000  4.900000
2018-01-15  3.500000  4.787552       NaN  4.850000
2018-01-16       NaN  4.784953  4.750000       NaN
2018-01-17       NaN  4.798615  4.750000  4.850000
2018-01-18       NaN  4.848685  4.840103  4.850000
2018-01-19       NaN  4.868836  4.850000  4.850000
2018-01-22       NaN  4.844195  4.848148  4.850000
2018-01-23       NaN  4.897371  4.835000  4.858788
2018-01-24       NaN  4.893941  4.850000  4.850000
2018-01-25       NaN  4.883550  4.869459       NaN
2018-01-26       NaN  4.820678 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:236: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:273: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:283: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Program

             日期   6个月存单利率      足年国股  7天质押加权利率  7天同存加权利率  承兑金额/亿元  贴现金额/亿元  \
508  2019-11-05  3.050000  2.803539    2.5211    3.0935   562.28   380.17   
509  2019-11-06  3.078033  2.759013    2.4829    2.9412   654.92   455.28   
510  2019-11-07  3.099352  2.711261    2.5245    2.9720   678.66   445.01   
511  2019-11-08       NaN  2.728587    2.5281    2.8276      NaN      NaN   
512  2019-11-11       NaN       NaN       NaN       NaN      NaN      NaN   

       收票    出票  
508  66.0  49.0  
509  99.0  74.0  
510  92.0  93.0  
511  31.0  48.0  
512  33.0  30.0  


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

          index    足年国股f   预测足年国股 足年国股fq2 预测涨跌    预测涨跌2
0    2018-01-12        5  5.23249                      
1    2018-01-15     5.04  5.26485                      
2    2018-01-16     5.05  5.35976                      
3    2018-01-17     5.12  5.32702                      
4    2018-01-18     5.35  5.28718                      
5    2018-01-19      5.4  5.31551                      
6    2018-01-22     5.16  5.41479                      
7    2018-01-23      5.1  5.43321                      
8    2018-01-24     5.13  5.43796                      
9    2018-01-25     5.18  5.42416                      
10   2018-01-26      5.3  5.45227                      
11   2018-01-29      5.2  5.46231                      
12   2018-02-02      5.1  5.45625                      
13   2018-02-05      5.1  5.43683                      
14   2018-02-06     5.15  5.43768                      
15   2018-02-07      5.1  5.45718       0    0         
16   2018-02-08     5.08  5.41304       0    0  

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:503: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:512: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

